In [80]:
import pandas as pd
import ast
import numpy as np

Сборка датасета

In [ ]:
df_list =[]

In [ ]:
ginekologs_1 = pd.read_csv('/content/data_ginekolog_0_2000.csv')
df_list.append(ginekologs_1)
ginekologs_2 = pd.read_csv('/content/data_ginekolog_2000_3397.csv')
df_list.append(ginekologs_2)
ginekologs_3 = pd.read_csv('/content/data_ginekolog_3397_3500.csv')
df_list.append(ginekologs_3)
ginekologs_4 = pd.read_csv('/content/data_ginekolog_3500_4500.csv')
df_list.append(ginekologs_4)
ginekologs_5 = pd.read_csv('/content/data_ginekolog_4500_5500.csv')
df_list.append(ginekologs_5)
ginekologs_6 = pd.read_csv('/content/data_ginekolog_5500_6982.csv')
df_list.append(ginekologs_6)

In [ ]:
kardiologs_1 = pd.read_csv('/content/data_kardiologs_0_1000.csv')
df_list.append(kardiologs_1)
kardiologs_2 = pd.read_csv('/content/data_kardiologs_1000_2500.csv')
df_list.append(kardiologs_2)
kardiologs_3 = pd.read_csv('/content/data_kardiologs_2500_3695.csv')
df_list.append(kardiologs_3)

In [ ]:
nevrologs_1 = pd.read_csv('/content/data_nevrologs_0 2500.csv')
df_list.append(nevrologs_1)
nevrologs_2 = pd.read_csv('/content/data_nevrologs_2500_5120.csv')
df_list.append(nevrologs_2)

In [ ]:
onkologs_1 = pd.read_csv('/content/data_onkologi_0_1395.csv')
df_list.append(onkologs_1)
onkologs_2 = pd.read_csv('/content/data_onkologi_1395_2789.csv')
df_list.append(onkologs_2)

In [ ]:
terpevts_1 = pd.read_csv('/content/data_terpevt_0_500.csv')
df_list.append(terpevts_1)
terpevts_2 = pd.read_csv('/content/data_terpevt_500_1000.csv')
df_list.append(terpevts_2)
terpevts_3 = pd.read_csv('/content/data_terpevt_1000_1500.csv')
df_list.append(terpevts_3)
terpevts_4 = pd.read_csv('/content/data_terpevt_1500_2000.csv')
df_list.append(terpevts_4)
terpevts_5 = pd.read_csv('/content/data_terpevt_2000_3000.csv')
df_list.append(terpevts_5)
terpevts_6 = pd.read_csv('/content/data_terpevt_3000_4000.csv')
df_list.append(terpevts_6)
terpevts_7 = pd.read_csv('/content/data_terpevt_4000_4500.csv')
df_list.append(terpevts_7)
terpevts_8 = pd.read_csv('/content/data_terpevt_4500_5000.csv')
df_list.append(terpevts_8)
terpevts_9 = pd.read_csv('/content/data_terpevt_5000_5500.csv')
df_list.append(terpevts_9)
terpevts_10 = pd.read_csv('/content/data_terpevt_5500_6500.csv')
df_list.append(terpevts_10)
terpevts_11 = pd.read_csv('/content/data_terpevt_6500_7500.csv')
df_list.append(terpevts_11)
terpevts_12 = pd.read_csv('/content/data_terpevt_7500_8500.csv')
df_list.append(terpevts_12)
terpevts_13 = pd.read_csv('/content/data_terpevt_8500_9500.csv')
df_list.append(terpevts_13)
terpevts_14 = pd.read_csv('/content/data_terpevt_9500_10500.csv')
df_list.append(terpevts_14)
terpevts_15 = pd.read_csv('/content/data_terpevt_10500_11090.csv')
df_list.append(terpevts_15)

In [ ]:
endokrinologs_1 = pd.read_csv('/content/data_endokrinolog_0_1274.csv')
df_list.append(endokrinologs_1)
endokrinologs_2 = pd.read_csv('/content/data_endokrinolog_1274_2549.csv')
df_list.append(endokrinologs_2)

In [ ]:
combined_df = pd.concat(df_list, ignore_index=True)
combined_df.to_csv('prodoctorov_data1.csv')

Подготовка к EDA

In [241]:
data = pd.read_csv('/content/prodoctorov_data1.csv')

In [242]:
data['clinics_count'] = 0
data['clinics_parsed'] = data['clinics'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])
data['clinics_count'] = data['clinics_parsed'].apply(len)

for i in range(1, 3 + 1):
    data[f'clinic_{i}_name'] = data['clinics_parsed'].apply(lambda x: x[i-1]['name'] if len(x) >= i else None)
    data[f'clinic_{i}_address'] = data['clinics_parsed'].apply(lambda x: x[i-1]['address'] if len(x) >= i else None)
    data[f'clinic_{i}_metro'] = data['clinics_parsed'].apply(lambda x: x[i-1]['metro'] if len(x) >= i else None)

data = data.drop(['clinics', 'clinics_parsed'], axis=1)

In [243]:
data['link'] = data['link'].str.split('#').str[0]

In [244]:
reviews_data = data[['name', 'link', 'reviews']].copy()

def parse_reviews(reviews):
    try:
        if pd.isna(reviews) or reviews == 'No value':
            return []
        return ast.literal_eval(reviews)
    except:
        return []

reviews_data['reviews_parsed'] = reviews_data['reviews'].apply(parse_reviews)
reviews_data = reviews_data[reviews_data['reviews_parsed'].apply(len) > 0]
reviews_exploded = reviews_data.explode('reviews_parsed')

reviews_data = pd.DataFrame({
    'doctor_name': reviews_exploded['name'],
    'doctor_link': reviews_exploded['link'],
    'rate': reviews_exploded['reviews_parsed'].apply(lambda x: x.get('rate')),
    'date': reviews_exploded['reviews_parsed'].apply(lambda x: x.get('date')),
    'comment': reviews_exploded['reviews_parsed'].apply(lambda x: x.get('comment')),
    'clinic': reviews_exploded['reviews_parsed'].apply(lambda x: x.get('clinic'))
})

In [245]:
reviews_data = reviews_data.reset_index(drop=True)

In [246]:
del data['reviews']

In [247]:
data = data.replace('No value', np.nan)

to_str_col = ['name', 'link', 'speciality', 'clinic_1_name', 'clinic_2_name', 'clinic_3_name', 'clinic_1_address', 'clinic_2_address', 'clinic_3_address']
for col in to_str_col:
  data[col] = data[col].astype('string')

to_cat_col = ['clinic_1_metro', 'clinic_2_metro', 'clinic_3_metro']
for col in to_cat_col:
  data[col] = data[col].astype('category')

data['rating'] = data['rating'].astype('float64')

data['price'] = data['price'].astype('float64')

to_num_col = ['price', 'experience', 'review_count','clinics_count']
for col in to_num_col:
  data[col] = data[col].astype('Int64')


Доктора

In [248]:
data = data.drop_duplicates(subset=['link'])


In [249]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29760 entries, 0 to 31208
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   name              29760 non-null  string  
 1   link              29760 non-null  string  
 2   speciality        29760 non-null  string  
 3   price             5933 non-null   Int64   
 4   experience        25851 non-null  Int64   
 5   rating            17357 non-null  float64 
 6   review_count      29760 non-null  Int64   
 7   is_kids           29760 non-null  bool    
 8   is_adults         29760 non-null  bool    
 9   clinics_count     29760 non-null  Int64   
 10  clinic_1_name     29760 non-null  string  
 11  clinic_1_address  29760 non-null  string  
 12  clinic_1_metro    28454 non-null  category
 13  clinic_2_name     9218 non-null   string  
 14  clinic_2_address  9218 non-null   string  
 15  clinic_2_metro    8301 non-null   category
 16  clinic_3_name     2712 non-

In [250]:
data.describe()

,price,experience,rating,review_count,clinics_count
count,5933.0,25851.0,17357.000000,29760.0,29760.0
mean,4698.933592,20.611388,2.552653,6.545329,1.438172
std,4034.023685,15.162585,1.546338,17.424247,0.7847
min,500.0,1.0,0.000000,0.0,1.0
25%,2800.0,9.0,1.400000,0.0,1.0
50%,3500.0,19.0,2.500000,2.0,1.0
75%,5400.0,30.0,3.900000,6.0,2.0
max,150000.0,300.0,5.000000,627.0,13.0


In [256]:
data = data[data['experience']<300.0]

In [257]:
data = data[(data['price'].notna())|(data['experience'].notna())|(data['rating'].notna())|(data['review_count']!= 0)]

In [258]:
data.describe()

,price,experience,rating,review_count,clinics_count
count,5925.0,25829.0,17333.000000,25829.0,25829.0
mean,4699.791224,20.373417,2.554659,7.536025,1.482094
std,4036.159258,12.788833,1.545841,18.503138,0.816124
min,500.0,1.0,0.000000,0.0,1.0
25%,2800.0,9.0,1.500000,0.0,1.0
50%,3500.0,19.0,2.500000,2.0,1.0
75%,5400.0,30.0,3.900000,8.0,2.0
max,150000.0,71.0,5.000000,627.0,13.0


In [259]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25829 entries, 0 to 31208
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   name              25829 non-null  string  
 1   link              25829 non-null  string  
 2   speciality        25829 non-null  string  
 3   price             5925 non-null   Int64   
 4   experience        25829 non-null  Int64   
 5   rating            17333 non-null  float64 
 6   review_count      25829 non-null  Int64   
 7   is_kids           25829 non-null  bool    
 8   is_adults         25829 non-null  bool    
 9   clinics_count     25829 non-null  Int64   
 10  clinic_1_name     25829 non-null  string  
 11  clinic_1_address  25829 non-null  string  
 12  clinic_1_metro    24586 non-null  category
 13  clinic_2_name     8731 non-null   string  
 14  clinic_2_address  8731 non-null   string  
 15  clinic_2_metro    7856 non-null   category
 16  clinic_3_name     2628 non-

ОТЗЫВЫ

In [260]:
reviews_data = reviews_data.drop_duplicates()

In [261]:
reviews_data = reviews_data.replace('No value', np.nan)

to_str_col = ['doctor_name', 'doctor_link','comment','clinic']
for col in to_str_col:
  reviews_data[col] = reviews_data[col].astype('string')

reviews_data['rate'] = pd.to_numeric(reviews_data['rate'], errors='coerce')

reviews_data['date'] = pd.to_datetime(reviews_data['date'], format='%Y-%m-%d')


/tmp/ipython-input-364684730.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  reviews_data = reviews_data.replace('No value', np.nan)


In [262]:
reviews_data['comment'] = reviews_data['comment'].str.replace('\u200b', '')

In [263]:
reviews_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 65048 entries, 0 to 69382
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   doctor_name  65048 non-null  string        
 1   doctor_link  65048 non-null  string        
 2   rate         63724 non-null  float64       
 3   date         65048 non-null  datetime64[ns]
 4   comment      64096 non-null  string        
 5   clinic       55065 non-null  string        
dtypes: datetime64[ns](1), float64(1), string(4)
memory usage: 3.5 MB


In [264]:
reviews_data['rate'].value_counts().head(10)

,count
rate,
5.0,51482
0.0,5406
3.8,1259
4.8,1040
1.3,727
0.2,587
0.5,505
0.8,359
4.5,358


In [265]:
reviews_data['date'].value_counts().head(10)

,count
date,
2025-10-29,154
2025-10-24,139
2025-10-31,135
2025-10-30,134
2025-10-15,131
2025-10-22,128
2025-10-23,121
2025-10-14,118
2025-10-08,117


In [273]:
reviews_data = reviews_data[(reviews_data['rate'].notna())|(reviews_data['comment'].notna())]

In [267]:
reviews_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64096 entries, 0 to 69382
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   doctor_name  64096 non-null  string        
 1   doctor_link  64096 non-null  string        
 2   rate         63724 non-null  float64       
 3   date         64096 non-null  datetime64[ns]
 4   comment      64096 non-null  string        
 5   clinic       54204 non-null  string        
dtypes: datetime64[ns](1), float64(1), string(4)
memory usage: 3.4 MB


In [268]:
reviews_data.describe()

,rate,date
count,63724.000000,64096
mean,4.311945,2022-05-08 17:56:40.499251456
min,0.000000,2011-09-02 00:00:00
25%,5.000000,2019-08-11 00:00:00
50%,5.000000,2023-04-11 00:00:00
75%,5.000000,2025-01-11 00:00:00
max,5.000000,2025-11-04 00:00:00
std,1.611258,NaN


In [271]:
data.to_csv('data_prodoctorov_to_eda.csv')

In [272]:
reviews_data.to_csv('data_prodoctorov_reviews_to_eda.csv')